In [3]:
# benchmark_vassoura.ipynb – Comparação entre versão modular (VassouraSession)
# e abordagem direta com recomputação completa

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time

from vassoura.core import VassouraSession

# Gerar dataset sintético grande
np.random.seed(42)
n = 1_000_000
X = pd.DataFrame({
    f"x{i}": np.random.randn(n) for i in range(15)
})

# Introduzir correlações artificiais
X["x1"] = X["x0"] * 0.9 + np.random.normal(0, 0.1, n)
X["x2"] = X["x0"] * 0.8 + X["x1"] * 0.1
X["x3"] = X["x4"] * 0.9 + np.random.normal(0, 0.05, n)

# Adicionar coluna target binária
X["target"] = (X["x0"] + np.random.randn(n) > 0).astype(int)

In [4]:
# ------------------------------------------------------------
# Abordagem tradicional (recomputa tudo sempre)
# ------------------------------------------------------------
start = time.time()
vs = VassouraSession(
    X,
    target_col="target",
    
    heuristics=[
        #"corr",
        #"vif",
        #"iv",
        "graph_cut"
    ],
    
    thresholds={
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,   # limiar para graph-cut
    },
)
df_limpo = vs.run(recompute=True)

end = time.time()
print(f"Tempo com recomputação total: {end - start:.2f}s")

[Vassoura] IV heuristic (thr<0.01) – removendo low IV
  → dropped ['x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14'] (iv<0.01)
Tempo com recomputação total: 21.10s


c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\site-packages\vassoura\core.py:245: UserWarning: Graph-cut heuristic placeholder – não implementada.
  warnings.warn("Graph-cut heuristic placeholder – não implementada.")


In [ ]:
# ------------------------------------------------------------
# Abordagem incremental com cache
# ------------------------------------------------------------
vs_cached = VassouraSession(
    X,
    target_col="target",
    heuristics=["corr", "vif", "iv"],
    thresholds={
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01
        }
    )

start = time.time()
for _ in range(3):
    _ = vs_cached.run(recompute=False)
end = time.time()
print(f"Tempo com reaproveitamento de caches: {end - start:.2f}s")

In [ ]:
# ------------------------------------------------------------
# Exibir variáveis removidas e histórico final
# ------------------------------------------------------------
print("\nColunas removidas:", vs_cached.dropped)
print("\nHistórico resumido:")
for step in vs_cached.history:
    print(" -", step)


In [ ]:
# Agora, em repetições subsequentes, só rodar corr+iv (pulando vif se não restarem variáveis)
start = time.time()
for _ in range(3):
    # Se X estiver com poucas colunas, evitar executar todo verbo "vif"
    remaining = len(vs_cached.df_current.columns) - 1  # menos alvo
    heur = ["corr", "iv"] if remaining > 1 else ["corr"]
    vs_cached.heuristics = heur
    _ = vs_cached.run(recompute=False)
end = time.time()
print(f"Tempo com reaproveitamento de caches (corr+iv): {end - start:.2f}s")


In [ ]:

# ------------------------------------------------------------
# Exibir variáveis removidas e histórico final
# ------------------------------------------------------------
print("\nColunas removidas:", vs_cached.dropped)
print("\nHistórico resumido:")
for step in vs_cached.history:
    print(" -", step)